In [2]:
# Importing Required Libraries
import os
import cv2
import numpy as np
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
from torchvision.models.detection import fasterrcnn_resnet50_fpn
from torch import nn
import pytesseract
from torchvision import models
import matplotlib.pyplot as plt

In [3]:
# Define Dataset Paths
TRAIN1_PATH = r"C:\Users\chara\Downloads\licpr\Licplatesdetection_train1"  # Vehicle images with bounding box annotations
TRAIN1_ANNOTATIONS = r"C:\Users\chara\Downloads\licpr\Licplatesdetection_train1_annotations.csv"  # Annotations for Training Set 1

TRAIN2_PATH = r"C:\Users\chara\Downloads\licpr\Licplatesrecognition_train2"  # License plate images with text annotations
TRAIN2_ANNOTATIONS = r"C:\Users\chara\Downloads\licpr\Licplatesrecognition_train2_annotations.csv"  # Text annotations for Training Set 2

TEST_PATH = r"C:\Users\chara\Downloads\licpr\test" # Test set images

In [4]:
# Custom Dataset for Training Set 1
class LicensePlateDataset(Dataset):
    def __init__(self, image_dir, annotations_file, transform=None):
        """
        image_dir: Directory containing the images.
        annotations_file: CSV file with bounding box annotations.
        transform: Data augmentation or transformations.
        """
        self.image_dir = image_dir
        self.annotations = pd.read_csv(annotations_file)
        self.transform = transform

    def __len__(self):
        return len(self.annotations)

    def __getitem__(self, idx):
        # Load image and annotations
        img_path = os.path.join(self.image_dir, self.annotations.iloc[idx, 0])
        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        boxes = self.annotations.iloc[idx, 1:5].values.astype('float32').tolist()
        labels = torch.ones((1,), dtype=torch.int64)  # Assuming single class (license plate)

        # Apply transformations
        if self.transform:
            image = self.transform(image)

        target = {
            "boxes": torch.tensor([boxes]),
            "labels": labels,
        }
        return image, target

In [5]:
# Build the License Plate Detection Model
def build_detection_model():
    """
    Creates a Faster R-CNN model pre-trained on the COCO dataset.
    """
    model = fasterrcnn_resnet50_fpn(pretrained=True)
    # Modify the classifier to detect only one class (license plate)
    num_classes = 2  # Background + License Plate
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    model.roi_heads.box_predictor = nn.Linear(in_features, num_classes)
    return model

# Initialize the model
detection_model = build_detection_model()
detection_model = detection_model.to('cpu')

C:\Users\chara\AppData\Roaming\Python\Python311\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\chara\AppData\Roaming\Python\Python311\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=FasterRCNN_ResNet50_FPN_Weights.COCO_V1`. You can also use `weights=FasterRCNN_ResNet50_FPN_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [6]:
# Custom Dataset for Training Set 2 (License Plate OCR)
class LicensePlateOCRDataset(Dataset):
    def __init__(self, image_dir, annotations_file, transform=None):
        """
        image_dir: Directory containing cropped license plate images.
        annotations_file: CSV file with license plate text annotations.
        transform: Data augmentation or transformations.
        """
        self.image_dir = image_dir
        self.annotations = pd.read_csv(annotations_file)
        self.transform = transform

    def __len__(self):
        return len(self.annotations)

    def __getitem__(self, idx):
        # Load image and corresponding text
        img_path = os.path.join(self.image_dir, self.annotations.iloc[idx, 0])
        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        label = self.annotations.iloc[idx, 1]

        # Apply transformations
        if self.transform:
            image = self.transform(image)

        return image, label

In [7]:
# Build OCR Model using Pre-trained ResNet
class LicensePlateOCR(nn.Module):
    def __init__(self, num_classes=36):  # 26 letters + 10 digits
        super(LicensePlateOCR, self).__init__()
        self.backbone = models.resnet18(pretrained=True)
        self.backbone.fc = nn.Linear(self.backbone.fc.in_features, num_classes)

    def forward(self, x):
        return self.backbone(x)

# Initialize the OCR model
ocr_model = LicensePlateOCR(num_classes=36)  # Adjust for alphanumeric output
ocr_model = ocr_model.to('cpu')

C:\Users\chara\AppData\Roaming\Python\Python311\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [8]:
# Training Function for Detection Model
def train_detection_model(model, dataloader, optimizer, num_epochs=10):
    model.train()
    for epoch in range(num_epochs):
        epoch_loss = 0
        for images, targets in dataloader:
            images = list(img.to('cpu') for img in images)
            targets = [{k: v.to('cpu') for k, v in t.items()} for t in targets]

            loss_dict = model(images, targets)
            losses = sum(loss for loss in loss_dict.values())

            optimizer.zero_grad()
            losses.backward()
            optimizer.step()

            epoch_loss += losses.item()
        print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {epoch_loss:.4f}")

# Training Function for OCR Model
def train_ocr_model(model, dataloader, criterion, optimizer, num_epochs=10):
    model.train()
    for epoch in range(num_epochs):
        epoch_loss = 0
        for images, labels in dataloader:
            images = images.to('cpu')
            labels = labels.to('cpu')

            outputs = model(images)
            loss = criterion(outputs, labels)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            epoch_loss += loss.item()
        print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {epoch_loss:.4f}")

In [9]:
import pandas as pd

# Inference Pipeline with Results Storage
def detect_and_recognize(image_paths, detection_model, ocr_model, output_excel_path):
    """
    Processes a list of image paths, detects license plates, recognizes characters, 
    and saves results in an Excel file.

    image_paths: List of paths to test images.
    detection_model: Pre-trained license plate detection model.
    ocr_model: Pre-trained OCR model.
    output_excel_path: Path to save the Excel file.
    """
    results = []  # To store results in a list

    for image_path in image_paths:
        # Read the image
        image = cv2.imread(image_path)
        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        transform = transforms.ToTensor()
        image_tensor = transform(image_rgb).unsqueeze(0).to('cpu')

        # Step 1: Detect License Plate
        detection_model.eval()
        with torch.no_grad():
            predictions = detection_model(image_tensor)

        # Extract bounding box if available
        if len(predictions[0]['boxes']) > 0:
            box = predictions[0]['boxes'][0].cpu().numpy().astype(int)
            cropped = image_rgb[box[1]:box[3], box[0]:box[2]]

            # Step 2: OCR Recognition
            ocr_model.eval()
            transform = transforms.Compose([
                transforms.ToPILImage(),
                transforms.Resize((224, 224)),
                transforms.ToTensor()
            ])
            cropped_tensor = transform(cropped).unsqueeze(0).to('cpu')

            with torch.no_grad():
                output = ocr_model(cropped_tensor)
                predicted_text = "".join(chr(c + ord('A')) for c in output.argmax(dim=1).cpu().numpy())
        else:
            predicted_text = "License plate not detected"

        # Append the result (image name and detected text)
        results.append({
            "Image Name": os.path.basename(image_path),
            "Detected Text": predicted_text
        })

    # Convert results to a DataFrame and save as Excel
    df_results = pd.DataFrame(results)
    df_results.to_excel(output_excel_path, index=False)
    print(f"Results saved to {output_excel_path}")

In [10]:
# List of test image paths
test_image_paths = [os.path.join(TEST_PATH, img) for img in os.listdir(TEST_PATH) if img.endswith(('.png', '.jpg', '.jpeg'))]

# Path to save the Excel file
output_excel_path = "license_plate_results.xlsx"

# Run the pipeline
detect_and_recognize(test_image_paths, detection_model, ocr_model, output_excel_path)


ValueError: too many values to unpack (expected 2)